# Predicting Forest Cover with Decision Trees

In [6]:
import org.apache.spark.sql.{functions => F}

import org.apache.spark.sql.{functions=>F}


## Preparing the Data

In [2]:
val data_path = "../data/covtype/covtype.data"

data_path = ../data/covtype/covtype.data


../data/covtype/covtype.data

In [3]:
val dataWithoutHeader = spark.read.option("header", "false").
    option("inferSchema", "true").csv(data_path)

dataWithoutHeader = [_c0: int, _c1: int ... 53 more fields]


[_c0: int, _c1: int ... 53 more fields]

In [4]:
val colNames = Seq(
        "Elevation", "Aspect", "Slope",
        "Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology",
        "Horizontal_Distance_To_Roadways",
        "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
        "Horizontal_Distance_To_Fire_Points"
      ) ++ (
        (0 until 4).map(i => s"Wilderness_Area_$i")
      ) ++ (
        (0 until 40).map(i => s"Soil_Type_$i")
      ) ++ Seq("Cover_Type")

colNames = List(Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Hillshade_9am, Hillshade_Noon, Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Wilderness_Area_0, Wilderness_Area_1, Wilderness_Area_2, Wilderness_Area_3, Soil_Type_0, Soil_Type_1, Soil_Type_2, Soil_Type_3, Soil_Type_4, Soil_Type_5, Soil_Type_6, Soil_Type_7, Soil_Type_8, Soil_Type_9, Soil_Type_10, Soil_Type_11, Soil_Type_12, Soil_Type_13, Soil_Type_14, Soil_Type_15, Soil_Type_16, Soil_Type_17, Soil_Type_18, Soil_Type_19, Soil_Type_20, Soil_Type_21, Soil_Type_22, Soil_Type_23, Soil_Type_24, Soil_Type_25, Soil_Type_26, Soil_Type_27, Soil_Type_28, Soil_Type_29, Soil_Type_30, Soil_Type_31, Soil_Type_32, Soil_Type_33, Soil_Type_34, Soil_...


List(Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Hillshade_9am, Hillshade_Noon, Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Wilderness_Area_0, Wilderness_Area_1, Wilderness_Area_2, Wilderness_Area_3, Soil_Type_0, Soil_Type_1, Soil_Type_2, Soil_Type_3, Soil_Type_4, Soil_Type_5, Soil_Type_6, Soil_Type_7, Soil_Type_8, Soil_Type_9, Soil_Type_10, Soil_Type_11, Soil_Type_12, Soil_Type_13, Soil_Type_14, Soil_Type_15, Soil_Type_16, Soil_Type_17, Soil_Type_18, Soil_Type_19, Soil_Type_20, Soil_Type_21, Soil_Type_22, Soil_Type_23, Soil_Type_24, Soil_Type_25, Soil_Type_26, Soil_Type_27, Soil_Type_28, Soil_Type_29, Soil_Type_30, Soil_Type_31, Soil_Type_32, Soil_Type_33, Soil_Type_34, Soil_Type_35, Soil_Type_36, Soil_Type_37, Soil_Type_38, Soil_Type_39, Cover_Type)

In [7]:
val data = dataWithoutHeader.toDF(colNames:_*).withColumn("Cover_Type", F.col("Cover_type").cast("double"))

data = [Elevation: int, Aspect: int ... 53 more fields]


[Elevation: int, Aspect: int ... 53 more fields]

In [8]:
data.head

[2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5.0]

# A First Decision Tree

In [10]:
val Array(trainData, testData) = data.randomSplit(Array(0.9, 0.1))
trainData.cache()
testData.cache()

trainData = [Elevation: int, Aspect: int ... 53 more fields]
testData = [Elevation: int, Aspect: int ... 53 more fields]


[Elevation: int, Aspect: int ... 53 more fields]

In [11]:
import org.apache.spark.ml.feature.VectorAssembler

In [12]:
val inputCols = trainData.columns.filter(_ != "Cover_Type")
val assembler = new VectorAssembler().setInputCols(inputCols)
    .setOutputCol("featureVector")

inputCols = Array(Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Hillshade_9am, Hillshade_Noon, Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Wilderness_Area_0, Wilderness_Area_1, Wilderness_Area_2, Wilderness_Area_3, Soil_Type_0, Soil_Type_1, Soil_Type_2, Soil_Type_3, Soil_Type_4, Soil_Type_5, Soil_Type_6, Soil_Type_7, Soil_Type_8, Soil_Type_9, Soil_Type_10, Soil_Type_11, Soil_Type_12, Soil_Type_13, Soil_Type_14, Soil_Type_15, Soil_Type_16, Soil_Type_17, Soil_Type_18, Soil_Type_19, Soil_Type_20, Soil_Type_21, Soil_Type_22, Soil_Type_23, Soil_Type_24, Soil_Type_25, Soil_Type_26, Soil_Type_27, Soil_Type_28, Soil_Type_29, Soil_Type_30, Soil_Type_31, Soil_Type_32, Soil_Type_33, Soil_Type_34, S...


[Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Hillshade_9am, Hillshade_Noon, Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Wilderness_Area_0, Wilderness_Area_1, Wilderness_Area_2, Wilderness_Area_3, Soil_Type_0, Soil_Type_1, Soil_Type_2, Soil_Type_3, Soil_Type_4, Soil_Type_5, Soil_Type_6, Soil_Type_7, Soil_Type_8, Soil_Type_9, Soil_Type_10, Soil_Type_11, Soil_Type_12, Soil_Type_13, Soil_Type_14, Soil_Type_15, Soil_Type_16, Soil_Type_17, Soil_Type_18, Soil_Type_19, Soil_Type_20, Soil_Type_21, Soil_Type_22, Soil_Type_23, Soil_Type_24, Soil_Type_25, Soil_Type_26, Soil_Type_27, Soil_Type_28, Soil_Type_29, Soil_Type_30, Soil_Type_31, Soil_Type_32, Soil_Type_33, Soil_Type_34, Soil_Type_35, Soil_Type_36, Soil_Type_37, Soil_Type_38, Soil_Type_39]

In [13]:
val assembledTrainData = assembler.transform(trainData)

assembledTrainData = [Elevation: int, Aspect: int ... 54 more fields]


[Elevation: int, Aspect: int ... 54 more fields]

In [14]:
assembledTrainData.select("featureVector").show(truncate = false)

+-----------------------------------------------------------------------------------------------------+
|featureVector                                                                                        |
+-----------------------------------------------------------------------------------------------------+
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1863.0,37.0,17.0,120.0,18.0,90.0,217.0,202.0,115.0,769.0,1.0,1.0])  |
|(54,[0,1,2,5,6,7,8,9,13,18],[1874.0,18.0,14.0,90.0,208.0,209.0,135.0,793.0,1.0,1.0])                 |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1879.0,28.0,19.0,30.0,12.0,95.0,209.0,196.0,117.0,778.0,1.0,1.0])   |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1888.0,33.0,22.0,150.0,46.0,108.0,209.0,185.0,103.0,735.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,14],[1889.0,28.0,22.0,150.0,23.0,120.0,205.0,185.0,108.0,759.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1889.0,353.0,30.0,95.0,39.0,67.0,153.0,172.0,146.0,600.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1896.0,337.0,12.0,30.0,6.0,175

In [15]:
import org.apache.spark.ml.classification.DecisionTreeClassifier

In [16]:
val classifier = new DecisionTreeClassifier().
    setSeed(42).
    setLabelCol("Cover_Type").
    setFeaturesCol("featureVector").
    setPredictionCol("prediction")


classifier = dtc_64312fa273bd


dtc_64312fa273bd

In [17]:
val model = classifier.fit(assembledTrainData)

model = DecisionTreeClassificationModel (uid=dtc_64312fa273bd) of depth 5 with 63 nodes


DecisionTreeClassificationModel (uid=dtc_64312fa273bd) of depth 5 with 63 nodes

In [18]:
println(model.toDebugString)

DecisionTreeClassificationModel (uid=dtc_64312fa273bd) of depth 5 with 63 nodes
  If (feature 0 <= 3052.5)
   If (feature 0 <= 2563.5)
    If (feature 10 <= 0.5)
     If (feature 0 <= 2451.5)
      If (feature 3 <= 15.0)
       Predict: 4.0
      Else (feature 3 > 15.0)
       Predict: 3.0
     Else (feature 0 > 2451.5)
      If (feature 17 <= 0.5)
       Predict: 2.0
      Else (feature 17 > 0.5)
       Predict: 3.0
    Else (feature 10 > 0.5)
     If (feature 22 <= 0.5)
      If (feature 9 <= 4598.0)
       Predict: 2.0
      Else (feature 9 > 4598.0)
       Predict: 2.0
     Else (feature 22 > 0.5)
      If (feature 9 <= 1102.0)
       Predict: 2.0
      Else (feature 9 > 1102.0)
       Predict: 2.0
   Else (feature 0 > 2563.5)
    If (feature 0 <= 2942.5)
     If (feature 15 <= 0.5)
      If (feature 17 <= 0.5)
       Predict: 2.0
      Else (feature 17 > 0.5)
       Predict: 3.0
     Else (feature 15 > 0.5)
      If (feature 9 <= 1357.5)
       Predict: 3.0
      Else (feature 9 >

In [19]:
model.featureImportances.toArray.zip(inputCols).sorted.reverse.foreach(println)

(0.778427540175678,Elevation)
(0.040561444120305205,Horizontal_Distance_To_Hydrology)
(0.03146363027227748,Hillshade_Noon)
(0.03097607213882755,Wilderness_Area_0)
(0.029083349083037755,Soil_Type_3)
(0.027350706263280768,Soil_Type_31)
(0.021927503190540414,Soil_Type_1)
(0.011332055405466444,Wilderness_Area_2)
(0.009588192084301072,Soil_Type_28)
(0.006053830447010267,Soil_Type_22)
(0.00571816142375842,Horizontal_Distance_To_Roadways)
(0.003133165481742783,Wilderness_Area_1)
(0.0022908420609322345,Hillshade_9am)
(0.001805882250606783,Horizontal_Distance_To_Fire_Points)
(2.8762560223494985E-4,Soil_Type_8)
(0.0,Wilderness_Area_3)
(0.0,Vertical_Distance_To_Hydrology)
(0.0,Soil_Type_9)
(0.0,Soil_Type_7)
(0.0,Soil_Type_6)
(0.0,Soil_Type_5)
(0.0,Soil_Type_4)
(0.0,Soil_Type_39)
(0.0,Soil_Type_38)
(0.0,Soil_Type_37)
(0.0,Soil_Type_36)
(0.0,Soil_Type_35)
(0.0,Soil_Type_34)
(0.0,Soil_Type_33)
(0.0,Soil_Type_32)
(0.0,Soil_Type_30)
(0.0,Soil_Type_29)
(0.0,Soil_Type_27)
(0.0,Soil_Type_26)
(0.0,Soil_Ty

In [20]:
val predictions = model.transform(assembledTrainData)
predictions.select("Cover_Type", "prediction", "probability").show(truncate = false)

+----------+----------+-------------------------------------------------------------------------------------------------+
|Cover_Type|prediction|probability                                                                                      |
+----------+----------+-------------------------------------------------------------------------------------------------+
|6.0       |3.0       |[0.0,0.0,0.033742225887365564,0.6317905370580352,0.04978868157921864,0.0,0.28467855547538057,0.0]|
|6.0       |4.0       |[0.0,0.0,0.04389395914819644,0.2842242503259452,0.43415906127770537,0.0,0.23772272924815296,0.0] |
|6.0       |3.0       |[0.0,0.0,0.033742225887365564,0.6317905370580352,0.04978868157921864,0.0,0.28467855547538057,0.0]|
|6.0       |3.0       |[0.0,0.0,0.033742225887365564,0.6317905370580352,0.04978868157921864,0.0,0.28467855547538057,0.0]|
|6.0       |3.0       |[0.0,0.0,0.033742225887365564,0.6317905370580352,0.04978868157921864,0.0,0.28467855547538057,0.0]|
|6.0       |3.0       |[

predictions = [Elevation: int, Aspect: int ... 57 more fields]


[Elevation: int, Aspect: int ... 57 more fields]

In [21]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

In [22]:
val evaluator = new MulticlassClassificationEvaluator().
    setLabelCol("Cover_Type").
    setPredictionCol("prediction")

evaluator = mcEval_dc150c77fec1


mcEval_dc150c77fec1

In [23]:
evaluator.setMetricName("accuracy").evaluate(predictions)

0.6999359250227132

In [24]:
evaluator.setMetricName("f1").evaluate(predictions)

0.6823944459410305

In [25]:
import org.apache.spark.mllib.evaluation.MulticlassMetrics

In [26]:
val predictionRDD = predictions.select("prediction", "Cover_Type").as[(Double, Double)].rdd

predictionRDD = MapPartitionsRDD[96] at rdd at <console>:43


MapPartitionsRDD[96] at rdd at <console>:43

In [27]:
val multiclassMetrics = new MulticlassMetrics(predictionRDD)

multiclassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@dc859ea


org.apache.spark.mllib.evaluation.MulticlassMetrics@dc859ea

In [28]:
multiclassMetrics.confusionMatrix

129136.0  55932.0   164.0    0.0    0.0  0.0  5379.0   
50005.0   199738.0  4250.0   101.0  0.0  0.0  780.0    
0.0       5843.0    25723.0  654.0  0.0  0.0  0.0      
0.0       19.0      1449.0   999.0  0.0  0.0  0.0      
43.0      7824.0    687.0    0.0    0.0  0.0  0.0      
0.0       6106.0    8975.0   547.0  0.0  0.0  0.0      
7887.0    182.0     54.0     0.0    0.0  0.0  10348.0  

In [30]:
val confusionMatrix = predictions.select("prediction", "Cover_Type")
    .groupBy("Cover_Type").pivot("prediction").count().na.fill(0).orderBy("Cover_Type")

confusionMatrix = [Cover_Type: double, 1.0: bigint ... 4 more fields]


[Cover_Type: double, 1.0: bigint ... 4 more fields]

In [31]:
confusionMatrix.show()

+----------+------+------+-----+---+-----+
|Cover_Type|   1.0|   2.0|  3.0|4.0|  7.0|
+----------+------+------+-----+---+-----+
|       1.0|129136| 55932|  164|  0| 5379|
|       2.0| 50005|199738| 4250|101|  780|
|       3.0|     0|  5843|25723|654|    0|
|       4.0|     0|    19| 1449|999|    0|
|       5.0|    43|  7824|  687|  0|    0|
|       6.0|     0|  6106| 8975|547|    0|
|       7.0|  7887|   182|   54|  0|10348|
+----------+------+------+-----+---+-----+



In [33]:
import org.apache.spark.sql.DataFrame

In [34]:
def classProbabilities(data: DataFrame): Array[Double] = {
    val total = data.count()
    data.groupBy("Cover_Type").count().
      orderBy("Cover_Type").
      select("count").as[Double].
      map(_ / total).
      collect()
  }

classProbabilities: (data: org.apache.spark.sql.DataFrame)Array[Double]


In [35]:
val trainPriorProbabilities = classProbabilities(trainData)
val testPriorProbabilities = classProbabilities(testData)
trainPriorProbabilities.zip(testPriorProbabilities).map{
    case(trainProb, cvProb) => trainProb * cvProb
}.sum

trainPriorProbabilities = Array(0.3645789700186487, 0.4874939033137283, 0.061626739348730454, 0.004718596088557357, 0.01636111509587338, 0.029891455075790177, 0.03532922105867164)
testPriorProbabilities = Array(0.36484094385343807, 0.4885455514118274, 0.060735215769845495, 0.004812071424888721, 0.016137625242751818, 0.029886400742433875, 0.035042191554814646)


0.37733732374892875